In [1]:
import os
os.chdir('..')

In [2]:
import pandas as pd
import numpy as np

In [3]:
!pwd

/home/jupyter-kriach/md_github_lat/multigroupcode/multigroup-code


In [4]:
from bilevel.build_all_models import *
from bilevel.OnlineLinearExpert import OnlineLinearExpert
from bilevel.OnlineTreeExpert import OnlineHoeffdingTree
from bilevel.utils import ordinal_encoder

## 1a ridge implementable validate

In [7]:
A_t = np.load('./data_frames/nparrays/acsog_allstates2021_20230808-095906_unshuffled__undropped_.npy')

In [8]:
df_oh = pd.read_pickle('./data_frames/dense_acs_mm_oh.pkl')

In [9]:
df_oh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1569188 entries, 0 to 1630166
Columns: 898 entries, WKHP to RAC1P_9
dtypes: float64(3), uint8(895)
memory usage: 1.4 GB


In [10]:
df_oh

,WKHP,AGEP,PINCP,OCCP_10,OCCP_20,OCCP_40,OCCP_51,OCCP_52,OCCP_60,OCCP_101,...,MAR_5,RAC1P_1,RAC1P_2,RAC1P_3,RAC1P_4,RAC1P_5,RAC1P_6,RAC1P_7,RAC1P_8,RAC1P_9
0,0.295918,0.025316,0.039472,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,0.397959,0.037975,0.030967,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
2,0.173469,0.025316,0.030467,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,0.051020,0.215190,0.053479,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0.091837,0.025316,0.009455,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630162,0.397959,0.291139,0.399670,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1630163,0.397959,0.379747,0.299615,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1630164,0.397959,0.367089,0.092501,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1630165,0.295918,0.063291,0.060483,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [11]:
y_dat = df_oh['PINCP']

In [12]:
df_oh.drop('PINCP', axis=1, inplace=True)

In [13]:
X_dat = drop('PINCP', axis=1, inplace=True)

In [25]:
expridge = OnlineLinearExpert(X_dat, y_dat, l2_pen=1.0)

In [26]:
dir_name = './restruc_res/models/baseline'
filename =  'ridge_acsog'

In [28]:
b_ridgebase = build_baseline_alwayson(dir_name, filename, A_t, expridge)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 1569188/1569188 [23:07<00:00, 1131.00it/s]


In [18]:
len(b_ridgebase.expert.cumloss_groupwise)

12

In [31]:
import joblib
oridge_imp = joblib.load('./restruc_res/models/baseline/ridge_acsog20230822-001825.pkl')

In [32]:
for gnum in range(12):
  print(oridge_imp.cumloss_groupwise[gnum][-1])

31098.672186126223
22396.608111110658
37590.35674739708
3350.0869638965455
334.9671698399062
20.015414809307583
56.00525708187821
4901.493675804962
75.07590235612507
2203.9094416215803
4963.369724429311
53495.28029723699


In [35]:
y_dat[1]

0.03096703186752714

## 1b Anh validate linear experts acs og order

In [5]:
A_t = np.load('./data_frames/nparrays/acsog_allstates2021_20230808-095906_unshuffled__undropped_.npy')

In [6]:
A_t.shape

(1569188, 12)

In [7]:
df_oh = pd.read_pickle('./data_frames/dense_acs_mm_oh.pkl')
y_dat = df_oh['PINCP']
X_dat = df_oh.drop('PINCP', axis=1)

In [12]:
X_dat.head(), X_dat.shape

(       WKHP      AGEP  OCCP_10  OCCP_20  OCCP_40  OCCP_51  OCCP_52  OCCP_60  \
 0  0.295918  0.025316        0        0        0        0        0        0   
 1  0.397959  0.037975        0        0        0        0        0        0   
 2  0.173469  0.025316        0        0        0        0        0        0   
 3  0.051020  0.215190        0        0        0        0        0        0   
 4  0.091837  0.025316        0        0        0        0        0        0   
 
    OCCP_101  OCCP_102  ...  MAR_5  RAC1P_1  RAC1P_2  RAC1P_3  RAC1P_4  \
 0         0         0  ...      1        1        0        0        0   
 1         0         0  ...      1        0        1        0        0   
 2         0         0  ...      1        1        0        0        0   
 3         0         0  ...      0        1        0        0        0   
 4         0         0  ...      1        1        0        0        0   
 
    RAC1P_5  RAC1P_6  RAC1P_7  RAC1P_8  RAC1P_9  
 0        0        0  

In [13]:
experts = [OnlineLinearExpert(X_dat, y_dat, l2_pen=1.0) for _ in range(A_t.shape[1])]

In [15]:
dir_name = './restruc_res/models/Anh'
filename =  'ridge_validate_acsog'

In [16]:
b_Anh = build_Anh(dir_name, filename, A_t, experts)

100%|██████████| 1569188/1569188 [1:57:09<00:00, 223.24it/s] 


In [19]:
Anh = joblib.load('./restruc_res/models/Anh/ridge_validate_acsog20230822-015307.pkl')

In [20]:
len(Anh.cuml_loss_curve)

12

In [21]:
for gnum in range(12):
  print(Anh.cuml_loss_curve[gnum][-1])

30475.73819020837
22227.669286399614
37181.656706009446
3326.114008403669
325.81384868452835
19.450365142991945
52.192409915398414
4709.026106546749
73.4781794658253
2063.3343254242445
4952.341527015159
52703.407476607375


## 2a hoeffding tree implementable , Anh htree 

In [5]:
A_t = np.load('./data_frames/nparrays/acsog_allstates2021_20230808-095906_unshuffled__undropped_.npy')

In [6]:
df_notoh = pd.read_pickle("./data_frames/dense_acs_mm_notoh.pkl")
cat_cols_sig = ['OCCP', 'SCHL', 'ST', 'JWTRNS', 'DRAT', 'COW', 'SEX', \
       'RELSHIPP', 'POBP', 'ENG', 'MAR', 'RAC1P'] # significant features from the earlier analysis
numeric_cols = ['WKHP', 'AGEP', 'PINCP']
print(len(cat_cols_sig), len(numeric_cols))
df_notoh = ordinal_encoder(df_notoh, cat_cols_sig)

12 3


In [7]:
y_dat = df_notoh['PINCP']
df_notoh.drop('PINCP', axis=1, inplace=True)

In [11]:
# from collections import Counter
# Counter(df_notoh['SEX'])

Counter({0.0: 805503, 1.0: 763685})

In [8]:
X_datdict = df_notoh.to_dict(orient='records')

In [8]:
exphoefftree = OnlineHoeffdingTree(X_datdict, y_dat)

In [12]:
dir_name = './restruc_res/models/Anh'
filename =  'htree_acsog'

In [10]:
# b_htreebase = build_baseline_alwayson(dir_name, filename, A_t, exphoefftree) #implementable

100%|██████████| 1569188/1569188 [05:24<00:00, 4841.25it/s]


In [9]:
experts = [OnlineHoeffdingTree(X_datdict, y_dat) for _ in range(A_t.shape[1])]

In [ ]:
b_Anh = build_Anh(dir_name, filename, A_t, experts)

 38%|███▊      | 595776/1569188 [06:42<11:57, 1355.80it/s]

In [16]:
Anh = joblib.load('./restruc_res/models/Anh/htree_acsog20230822-103518.pkl')
for gnum in range(A_t.shape[1]):
  print(Anh.cuml_loss_curve[gnum][-1])

20796.476086940093
13572.113422487375
23982.11516858531
2368.7230439906803
234.61328231566043
16.968470012738823
37.65348410463236
3014.090240771034
56.295410141188036
1447.7136464136383
3210.4167630921515
34368.589509425794


In [11]:
# import joblib
# ohtree_imp = joblib.load('./restruc_res/models/baseline/htree_acsog20230822-013138.pkl')

In [12]:
# for gnum in range(12):
#   print(ohtree_imp.cumloss_groupwise[gnum][-1])

20745.077407449095
13810.410604688319
24103.089793125517
2402.4420325015653
237.78001319990227
14.85627330175781
36.72450443720022
3012.412629404053
54.10512373150721
1460.7881079631447
3233.2895344743274
34555.48801213991
